In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
###### change current path
import os
path = "/content/gdrive/MyDrive/NUS_IRS_Project/"
os.chdir(path)
!ls

data  data_gathering  Modelling  models


# Pre-Processing Resume Text Column to Prepare for Matching - final 

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline

import re
import datetime
from datetime import date
from time import strptime
import operator

###############################################################################################
## Working on Resume data
###############################################################################################

In [ ]:
# First reading my resume csv
resume = pd.read_csv('data/EDA_resume.csv')

In [ ]:
#initial info
resume.shape

(2484, 5)

In [ ]:
resume.head()

,ID,Resume_str,Category,name,file_name
0,16852973,hr administrator marketing associate hr admini...,HR,Garp Alspach,Garp_Alspach_resume.pdf
1,22323967,hr specialist us hr operations summary versati...,HR,Callie Haske,Callie_Haske_resume.pdf
2,33176873,hr director summary years experience recruitin...,HR,Guenevere Wannamaker,Guenevere_Wannamaker_resume.pdf
3,27018550,hr specialist summary dedicated driven dynamic...,HR,Barnie Palms,Barnie_Palms_resume.pdf
4,17812897,hr manager skill highlights hr skills hr depar...,HR,Prudy Prall,Prudy_Prall_resume.pdf


In [ ]:
resume.drop(resume[resume['Resume_str'].isnull()].index, axis=0, inplace=True)

In [ ]:
resume.isnull().sum()

ID            0
Resume_str    0
Category      0
name          0
file_name     0
dtype: int64

ID	
Resume_str	
Category	
name	
file_name

In [ ]:
resume['Category'] = resume['Category'].str.lower()
resume['Resume_str'] = resume['Resume_str'].map(lambda x: x.strip())

In [ ]:
df_resume = resume[['ID','Category']]
df_resume['resume_combo'] = resume['Category']+" "+resume['Resume_str']
df_resume.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Category,resume_combo
0,16852973,hr,hr hr administrator marketing associate hr adm...
1,22323967,hr,hr hr specialist us hr operations summary vers...
2,33176873,hr,hr hr director summary years experience recrui...
3,27018550,hr,hr hr specialist summary dedicated driven dyna...
4,17812897,hr,hr hr manager skill highlights hr skills hr de...


In [ ]:
docs = df_resume['resume_combo']
docs_sample = docs.head(10)
docs_sample

0    hr hr administrator marketing associate hr adm...
1    hr hr specialist us hr operations summary vers...
2    hr hr director summary years experience recrui...
3    hr hr specialist summary dedicated driven dyna...
4    hr hr manager skill highlights hr skills hr de...
5    hr hr generalist summary dedicated focused adm...
6    hr hr manager summary human resources manager ...
7    hr hr manager professional summary senior hr p...
8    hr hr specialist summary possess years experie...
9    hr hr clerk summary translates business vision...
Name: resume_combo, dtype: object

In [ ]:
#Import all the dependencies
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

import string

import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('ã¯æ’ëœ')
stopwords.append('\n')
stopwords.append('•')
#Transforms words to TFIDF
vectorizer = TfidfVectorizer(stop_words = stopwords)

index = 0
keys = {}

for rem in df_resume.itertuples() :
    key = rem[0]
    keys[key] = index
    index += 1

#Fit the vectorizer to the data
vectorizer.fit(df_resume['resume_combo'].fillna(''))

#Transform the data
tfidf_scores = vectorizer.transform(df_resume['resume_combo'].fillna(''))

print(tfidf_scores.shape)
print(df_resume.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ëœ'] not in stop_words.
  % sorted(inconsistent)


(2483, 37415)
(2483, 3)


In [ ]:
test = pd.DataFrame(tfidf_scores.toarray(), columns = vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
test.head()

,aa,aaa,aaae,aaahc,aaai,aaas,aac,aacn,aacsb,aade,...,zutano,zvm,zx,zxx,zynex,zynx,zypher,zyrtec,zyvox,zzxzx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating my Stopword list
#### As seen there are so many unwanted tokens like numbers,ïƒ¼ etc , I need to add them in "stop words" list to train model

In [ ]:
#getting list of all tokens
word_list = test.columns.tolist()

In [ ]:
##Getting a list of unwanted words as s_words and adding to stopwords
s_words =[]
for word in word_list:
    #print(word)
    if re.search("^\W|^\d",word):
        s_words.append(word)

In [ ]:
s_words.append('')        
from nltk.corpus import stopwords
stopword_set = set(stopwords.words('english'))
stopword_set = list(stopword_set)
stopword_set.extend(s_words)

In [ ]:
def preprocess(text):
    stop_words = stopword_set
    #0. split words by whitespace
    text = text.split()
    
    # 1. lower case
    text = [word.lower() for word in text]
    
    # 2. remove punctuations
    punc_table = str.maketrans('','',string.punctuation)
    text = [word.translate(punc_table) for word in text]
    
    # 3. remove stop words
    text = [word for word in text if word not in stop_words]
    
    return text

In [ ]:
tokenized_doc = []
doc = df_resume['resume_combo']
#doc = docs_sample
for d in doc:
    tokenized_doc.append(preprocess(d))
#tokenized_doc

In [ ]:
# Convert tokenized document into gensim formated tagged data
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]

In [ ]:
num_doc = len(tagged_data)
num_doc
#confirm length (should be 14428)
len(tokenized_doc)

2483

In [ ]:
#settings to show epoch progress
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
#settings to show epoch progress
class EpochLogger(CallbackAny2Vec):
    
    def __init__(self):
        self.epoch = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [ ]:
## Load saved doc2vec model
model= Doc2Vec.load("models/my_doc2vec.model")

In [ ]:
## Get vector value
vec = np.empty([2483,20])

for k,i in enumerate(tokenized_doc):
    
    #print(i)
    vector = model.infer_vector(i)
    vec[k] = vector

# reshape into 2D
new_arr = np.reshape(vec,(-1,20))

In [ ]:
rng = range(1, 21)
vec_df = pd.DataFrame(new_arr, columns=['vec_' + str(i) for i in rng])

In [ ]:
vec_df.head(5)

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,vec_11,vec_12,vec_13,vec_14,vec_15,vec_16,vec_17,vec_18,vec_19,vec_20
0,-0.439975,1.840168,0.297730,-2.516496,0.444102,0.996615,0.323182,0.469302,-1.306204,-1.398821,0.770349,-0.719928,-1.646259,-1.317602,-0.721896,-0.884862,0.354683,-0.514585,-0.329901,0.204269
1,-0.758077,0.244846,-0.115515,-2.500766,-0.512021,1.331365,-1.116393,0.028450,-1.251336,-2.170205,1.310075,-0.858518,-0.236034,-1.825453,-0.203004,-0.778714,-1.383295,-1.309409,-0.134993,-0.813979
2,-1.037689,1.657029,-0.115936,-2.441690,-0.932377,1.784235,-0.193735,0.442878,-2.501367,-0.074665,0.646280,-0.247877,-2.026806,-0.484951,-1.821919,-1.437760,-0.062110,0.004031,0.456452,-0.624781
3,0.220052,1.450823,-0.868382,-3.682528,0.950884,0.969832,1.314280,1.074755,-1.504844,0.415978,1.467488,-0.619841,-1.292397,1.356230,-2.134342,0.246203,0.760033,-0.567221,-0.228156,-0.531860
4,-0.773689,2.548777,-0.579622,-2.209147,-0.240961,1.308380,0.494653,0.383832,-2.845117,0.442517,1.220889,-0.629782,-1.502091,-0.503124,-2.130828,-1.119459,0.717784,0.061341,0.579723,-1.112323


In [ ]:
# concatenate and safe the resume csv file
con_resume_1 = pd.concat([resume, vec_df], axis=1)
con_resume_1.to_csv('data/vector_resume_20.csv', index=False)

In [4]:
import gensim
gensim.__version__

'3.6.0'